### Welcome to the second class on E-ID in this 3-class series
Before we start, make sure you have everything installed by following one of the outlined methods in the ReadMe file. 

### Purpose of this exercise
This jupyter notebook explains the following:
1. Basic E-ID example using the BBS cryptographic scheme
2. Selective Disclosure using BBS
3. Unlinkability using BBS and what role Zero-knowledge proofs play in it.
5. The previous points will use the **docknetwork crypto library**

---
## Technical overview
In this exercise, we will explore the BBS cryptographic scheme, and see how it differs from a classic RSA scheme as explored in the previous exercise.

For this, we will use a different typescript library https://github.com/docknetwork/crypto-wasm-ts


In [26]:
// We start by declaring our most important asset which is the E-ID Verifiable Credential we will be working with.

const E_ID_DATA = {
    name: "Jack Sparrow",
    age: 61,
    profession: "IT Manager"
}

// For this advanced library, we will also need to specify the schema of Credential we're working with.
// Specifying the schema will be useful later to get the correct encoding of each field.
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        age: { type: 'integer', minimum: 0, maximum: 120},
    }
}

In [27]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

## Issuer

In [28]:
// Generating a keypair once signature parameters are created.
import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES } from '@docknetwork/crypto-wasm-ts';

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

In [36]:
// Issing a credential based on the E-ID data based on the specified schema
import { CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'

const schema = CredentialSchema.essential();
schema.properties[SUBJECT_STR] = E_ID_SCHEMA

const builder = new BBSCredentialBuilder();
builder.schema = new CredentialSchema(schema);
builder.subject = E_ID_DATA

const credential = builder.sign(secretKey);
credential.verify(publicKey)
// credential
const credentialToSendToHolder = credential.toJSON()
credentialToSendToHolder

{
  cryptoVersion: '0.7.0',
  credentialSchema: {
    id: 'data:application/json;charset=utf-8,',
    type: 'JsonSchemaValidator2018',
    version: '0.5.0',
    details: '{"jsonSchema":{"$schema":"http://json-schema.org/draft-07/schema#","definitions":{"encryptableCompString":{"type":"string"},"encryptableString":{"type":"string"}},"properties":{"credentialSubject":{"properties":{"age":{"maximum":120,"minimum":0,"type":"integer"},"name":{"type":"string"},"profession":{"type":"string"}},"type":"object"},"proof":{"properties":{"type":{"type":"string"}},"type":"object"}},"type":"object"},"parsingOptions":{"defaultDecimalPlaces":0,"defaultMinimumDate":-17592186044415,"defaultMinimumInteger":-4294967295,"useDefaults":false}}'
  },
  credentialSubject: { name: 'Jack Sparrow', age: 61, profession: 'IT Manager' },
  proof: {
    type: 'Bls12381BBSSignatureDock2023',
    proofValue: '2STFYHrUxdDAqabNjManhLf1dRjBWUJfW9u1XPJwoJMubFRyAt2CKyHVv6DwVuUDMjAR2UWbcneRkRGjFQNvAVchjhtGRKN4ZvHH5ZKgdAofDt'


## The information below To be reviewed by Linus and Imad
**Notice** the structure above which includes a proofValue rather than a signature. Is is because 
all communication that happen between our issuers, holders, verifiers only include a proof of signature,
not the signature itself. This results in the fact that we can generate many different proofs of the same signature

## Credential Holder

In [81]:
// The holder loads the credential back from JSON
const receivedCredential = BBSCredential.fromJSON(credentialToSendToHolder)
// console.log(receivedCredential)

import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'
const presentationBuilder = new PresentationBuilder();
presentationBuilder.addCredential(receivedCredential, publicKey)
presentationBuilder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));

const presentation = presentationBuilder.finalize();
//console.log(presentation)

### sending data to Verifier
data: 
1. selected data to be revealed
2. Schema of the credential
3. Proof of signature

All these info and more are stored in the "presentation object". Note that this object is a completely serializable JSON object.

## Verifier

In [41]:
const receivedObjectOnVerifierSide = presentation.toJSON()
console.log(receivedObjectOnVerifierSide)  // Assume this was serialized and sent across an API to the verifier.
console.log("-------------------------")
console.log(presentation.spec.credentials[0])

{
  version: '0.9.0',
  context: undefined,
  nonce: null,
  spec: {
    credentials: [ [Object] ],
    attributeEqualities: [],
    boundedPseudonyms: {},
    unboundedPseudonyms: {}
  },
  proof: 'PqJrdz8W5DWFRnmCvbM5jYVnbVKvEg2ZesRviTMMaFp4kL1QgZS4qD65PKTNBsXMJf2wSrh62PfJk6MfHE1R3AZcDRrNwWVYri2cdQpUfhWV4cwe2AGcLkKwwioirK3iSBKLKeqgxJupHS8n71pagmMkgJgxrKfJ1NQAJV8ytPYUJuPeMYAP6KPM621veo7cZWa3HHfGV1qnraqbY6M9bS4mbEYVUyK42geyC8Z1BhiLwRbg639eXDUeGP2KfCyJmyMiVmFHF5RhT8tdrcipuS4XfeKqcLkpqvLwENh4X2UTQWmYqnSwNhWQEgLEvgnRkkNiTyyNmfeYyXYsKUTCCYzBnkpMVSwrev9F11F97LJkdxoqRTVUQ872LDeSZRPQELwunoxabdr8yiNdH6kKLrHU7skMSBpQdJMTPYoSpGMajitqSdMFDaRdw7ajWAwQb3jsUQoxtvy1idjuzCjYrSMJd9gUZLshPxxiyMm97a5YorXxjQTuHCgXnYyXhce6MSzKFfA5jQPXoVoJMKrFbJU7g5M9zsTnKNwxZcGwfon2fSbw4Z2GAiP87Jv8zswgGfyEz3'
}
-------------------------
{
  version: '0.7.0',
  schema: {
    details: '{"jsonSchema":{"$schema":"http://json-schema.org/draft-07/schema#","definitions":{"encryptableCompString":{"type":"string"},"encryptableStrin

In [80]:
// Verification of the data the verifier received is as easy as two lines of code!
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const recreatedPres = Presentation.fromJSON(receivedObjectOnVerifierSide)
recreatedPres.verify([publicKey], undefined, undefined, undefined, undefined, undefined)

// Now, we can go ahead and access the shared data!
//recreatedPres.spec.credentials[0].revealedAttributes

{ verified: true, error: undefined }


### Is this cryptographic scheme unlinkable?

We can know that by checking for two conditions

1. Is there a signature or a cryptopgraphic data that is shared and could used to identify the credential holder?
2. Is there a part of data hash or not hashed that was included in the presented data that could be used to identify the credential holder?

In [46]:
// To answer the first point, we look at the presented proof 
const proof1 = presentationBuilder.finalize()["proof"].hex
const proof2 = presentationBuilder.finalize()["proof"].hex

// By generating a proof multiple times, and comparing the proof in each of them,
// we see that they're always different making it difficult (Impossible?!) to correlate the different proofs.
console.log(proof1 === proof2)

false


In [68]:
// To answer the second point, we look at the data shared with the verifier.
// As opposed to the previous exercise where we had to share all the data all the time
// even though possibly in a hashed format, in this case, we only share the selected data.

presentation.spec.credentials[1]["revealedAttributes"]

// In this scenario, we see that only the data needed to be revealed is revealed resulting in unlinkability.

{
  credentialSubject: { name: 'Jack Sparrow', profession: 'IT Manager' }
}
